<a href="https://colab.research.google.com/github/MinhongW/text_generation/blob/main/fine_tune_t5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import json
import math
import platform
import sys
import tensorflow as tf

In [ ]:
! pip install datasets transformers rouge-score nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
print(f"Python Platform: {platform.platform()}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Python Platform: Linux-5.10.147+-x86_64-with-glibc2.31

Python 3.9.16 (main, Dec  7 2022, 01:11:51) 
[GCC 9.4.0]
Pandas 1.5.3
GPU is available


In [9]:
!git clone github.com/MinhongW/text_generation.git

fatal: repository 'github.com/MinhongW/text_generation.git' does not exist


In [ ]:
t1 = open('data/table_train.json')
t2 = open('data/table_desc_train.json')
t3 = open('data/paper_train.json')

v1 = open('data/table_val.json')
v2 = open('data/table_desc_val.json')
v3 = open('data/paper_val.json')

te1 = open('data/table_test.json')
te2 = open('data/table_desc_test.json')
te3 = open('data/paper_test.json')

FileNotFoundError: ignored

In [ ]:
from google.colab import files
uploaded = files.upload()

KeyboardInterrupt: ignored

In [ ]:
t1 = open('table_train.json')
t2 = open('table_desc_train.json')
t3 = open('paper_train.json')

v1 = open('table_val.json')
v2 = open('table_desc_val.json')
v3 = open('paper_val.json')

te1 = open('table_test.json')
te2 = open('table_desc_test.json')
te3 = open('paper_test.json')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
tables_train = json.load(t1)
descs_train = json.load(t2)
papers_train = json.load(t3)

tables_val = json.load(v1)
descs_val = json.load(v2)
papers_val = json.load(v3)

tables_test = json.load(te1)
descs_test = json.load(te2)
papers_test = json.load(te3)

In [ ]:
# double check the order in table file and desc file

for i in range(len(tables_train)):
    table_id1 = tables_train[i]['table_id_paper']
    table_id2 = descs_train[i]['table_id_paper']
    if table_id1 != table_id2:
               print('oops')

# Naive representation

Simply flatten T into a sequence ignoring its table structure by concatenating captions, headers, metrics and targeted cell values.

In [ ]:
def naive_representation(tables, descs):
    """
    Input_text is generated by naive representation of the tables.
    Each table is simply flattened into a sequence ignoring its table structure
    by concatenating captions, headers, metrics and targeted cell values.
    Target_text is the description of the corresponding table.
    Returns a df contains input_text and target_text
    
    """
    
    data = {'input_text':[],
           'target_text':[]}
    
    for i in range(len(tables)):
        table = tables[i]
        caption = table['table_id'] + ' ' + table['caption']
        row_names = ' '.join(' '.join(x) for x in table['row_headers'])
        col_names = ' '.join(' '.join(x) for x in table['column_headers'])
        metrics = ' '.join(table['metrics_type'])
        values = ' '.join(' '.join(x) for x in table['contents'])        
        tmp = [caption, row_names, col_names, metrics, values]
        text = ' '.join(tmp)
        
        desc = descs[i]['description']        
        
        data['input_text'].append(text)
        data['target_text'].append(desc)
    
    df = pd.DataFrame(data)      
    
    return df

In [ ]:
df_train = naive_representation(tables_train, descs_train)
df_val = naive_representation(tables_val, descs_val)
df_test = naive_representation(tables_test, descs_test)

In [ ]:
df_train.head()

,input_text,target_text
0,table_2 Comparison of different position featu...,Table 2 summarizes the performances of propose...
1,table_3 Pearson correlation values between hum...,Table 3 presents the correlation results for t...
2,table_4 Comparison between rationale models (m...,Results. Table 4 presents the results of our r...
3,table_2 Spearman’s rank correlation results on...,Table 2 shows the results of our contextdepend...
4,table_4 Examples of attention weights in diffe...,"From Table 4, we can find that in the first ho..."


In [ ]:
df_train['target_text'][3]

'Table 2 shows the results of our contextdependent sense embedding models on the SCWS dataset. In this table, ρ refers to the Spearman’s rank correlation and a higher value of ρ indicates better performance. The baseline performances are from Huang et al. (2012), Chen et al. (2014), Neelakantan et al. (2014), Li and Jurafsky (2015), Tian et al. (2014) and Bartunov et al. (2016). Here Ours + CBOW denotes our model with a CBOW based energy function and Ours + Skip-gram denotes our model with a Skip-gram based energy function. The results above the thick line are the models based on context clustering methods and the results below the thick line are the probabilistic models including ours. The similarity metrics of context clustering based models are AvgSim and AvgSimC proposed by Reisinger and Mooney (2010). Tian et al. (2014) propose two metrics Model_M and Model_W which are similar to our HardSim and SoftSim metrics. From Table 2, we can observe that our model outperforms the other pro

In [ ]:
from datasets import Dataset
ds_train = Dataset.from_pandas(df_train)
ds_val = Dataset.from_pandas(df_val)
ds_test = Dataset.from_pandas(df_test)

# T5 MODEL

In [ ]:
if torch.cuda.is_available():
    dev = torch.device("cuda:0")
    print("Running on the GPU")
else:
    dev = torch.device("cpu")
    print("Running on the CPU")

## Training with trainer API

In [ ]:
from transformers import AutoTokenizer

In [ ]:
model_checkpoint = "t5-small"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
def tokenize_function(examples):
    model_inputs = tokenizer(examples["input_text"], padding="max_length", truncation=True)
    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
      labels = tokenizer(examples["target_text"], padding="max_length", truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


ds_train_tokenized = ds_train.map(tokenize_function, batched=True)
ds_val_tokenized = ds_val.map(tokenize_function, batched=True)
ds_test_tokenized = ds_test.map(tokenize_function, batched=True)

Map:   0%|          | 0/1084 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:3596: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/136 [00:00<?, ? examples/s]

Map:   0%|          | 0/135 [00:00<?, ? examples/s]

In [ ]:
ds_train_tokenized

Dataset({
    features: ['input_text', 'target_text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1084
})

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer)

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [ ]:
batch_size = 4
model_name = "t5-test"
model_dir = f"model/{model_name}"

In [ ]:
args = Seq2SeqTrainingArguments(
    model_dir,
    evaluation_strategy="steps",
    eval_steps=100,
    logging_strategy="steps",
    logging_steps=100,
    save_strategy="steps",
    save_steps=200,
    learning_rate=4e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=15,
    predict_with_generate=True,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="rouge1",
    report_to="tensorboard"
)

In [ ]:
#!pip install evaluate

In [ ]:
import nltk
nltk.download('punkt')
import string

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import numpy as np

metric = load_metric("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip()))
                      for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) 
                      for label in decoded_labels]
    
    # Compute ROUGE scores
    result = metric.compute(predictions=decoded_preds, references=decoded_labels,
                            use_stemmer=True)

    # Extract ROUGE f1 scores
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length to metrics
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id)
                      for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
# Function that returns an untrained model to be trained
def model_init():
    return AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

trainer = Seq2SeqTrainer(
    model_init=model_init,
    args=args,
    train_dataset=ds_train_tokenized,
    eval_dataset=ds_val_tokenized,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
100,5.225300,1.645161,0.007000,0.000000,0.007000,0.007000,0.139700
200,1.684900,1.492557,0.007000,0.000000,0.007000,0.007000,0.139700
300,1.477800,1.447179,3.217600,1.237100,2.644300,3.005800,4.330900
400,1.456500,1.427104,12.872100,5.064300,10.392900,12.114100,14.669100
500,1.491400,1.411024,16.466000,6.772200,13.485500,15.562800,17.882400
600,1.342700,1.401811,17.740800,7.493300,14.545100,16.825400,19.000000
700,1.497500,1.389551,17.941600,7.330000,14.629700,16.885100,19.000000
800,1.397600,1.382576,17.881600,7.398400,14.435800,16.826500,19.000000
900,1.364600,1.374694,18.137600,7.551300,14.511200,17.047800,19.000000
1000,1.407100,1.368856,17.919700,7.557100,14.392900,16.809600,19.000000


TrainOutput(global_step=4065, training_loss=1.4533225553355387, metrics={'train_runtime': 1325.5435, 'train_samples_per_second': 12.267, 'train_steps_per_second': 3.067, 'total_flos': 2200657691934720.0, 'train_loss': 1.4533225553355387, 'epoch': 15.0})

In [ ]:
trainer.save_model()

In [ ]:
model_dir = f"model/{model_name}"

tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

max_input_length = 512

In [ ]:
df_test['input_text'][0]

'table_5 Link prediction results on the test-I, test-II, and test-all sets of FB122 and WN18 (filtered setting). FB122 TransE FB122 TransH FB122 TransR FB122 KALE-Trip FB122 KALE-Pre FB122 KALE-Joint WN18 TransE WN18 TransH WN18 TransR WN18 KALE-Trip WN18 KALE-Pre WN18 KALE-Joint Test-I MRR Test-I MED Test-I HITS@3 (%) Test-I HITS@5 (%) Test-I HITS@10 (%) Test-II MRR Test-II MED Test-II HITS@3 (%) Test-II HITS@5 (%) Test-II HITS@10 (%) Test-ALL MRR Test-ALL MED Test-ALL HITS@3 (%) Test-ALL HITS@5 (%) Test-ALL HITS@10 (%) MRR MED HITS@3 (%) HITS@5 (%) HITS@10 (%) MRR MED HITS@3 (%) HITS@5 (%) HITS@10 (%) MRR MED HITS@3 (%) HITS@5 (%) HITS@10 (%) 0.296 13.0 36.0 41.5 48.1 0.630 2.0 77.5 82.8 88.4 0.480 2.0 58.9 64.2 70.2 0.280 15.0 33.6 39.1 46.4 0.606 2.0 70.1 75.4 82.0 0.460 3.0 53.7 59.1 66.0 0.283 16.0 33.4 39.2 46.0 0.499 2.0 57.0 63.2 70.1 0.401 5.0 46.4 52.4 59.3 0.299 10.0 36.6 42.9 50.2 0.650 2.0 79.0 83.4 88.7 0.492 2.0 59.9 65.2 71.4 0.291 11.0 35.8 41.9 49.8 0.713 1.0 82.9 86

In [ ]:
input_text = df_test['input_text'][0]


input_ids = tokenizer.encode(input_text, truncation=True, return_tensors='pt')

# inputs = tokenizer(text, max_length=max_input_length, truncation=True, return_tensors="pt")
# output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=10, max_length=64)
# decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
# predicted_title = nltk.sent_tokenize(decoded_output.strip())[0]
output = model.generate(input_ids=input_ids, max_length=512)
tokenizer.decode(output[0])

'<pad> Table 5 shows the results of the test-I, test-II, and test-all sets of FB122 and WN18 (filtered setting). The results of the test-I, test-II, and test-all sets of FB122 and WN18 (filtered setting) are compared to the test-I, test-II, and test-all sets of FB122 and WN18 (filtered setting). The results of the test-II set are compared to the test-I set of FB122 and KALE-Joint.</s>'

## Old code

In [ ]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, Adafactor

In [ ]:
# need to double check here, do we need to use prefix for t5
model_checkpoint = 't5-small'
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

In [ ]:
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small', return_dict=True)
#moving the model to GPU
model.to(dev)

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [ ]:
batch_size=4
num_of_batches=math.ceil(len(df_train)/batch_size)
num_of_batches

271

In [ ]:
# Initiating the Adafactor optimizer with recommended T5 settings
optimizer = Adafactor(model.parameters(),lr=1e-3,
                      eps=(1e-30, 1e-3),
                      clip_threshold=1.0,
                      decay_rate=-0.8,
                      beta1=None,
                      weight_decay=0.0,
                      relative_step=False,
                      scale_parameter=False,
                      warmup_init=False)

In [ ]:
# Html based progress bar

from IPython.display import HTML, display
def progress(loss,value, max=100):
    return HTML(""" Batch loss :{loss}
      <progress    
    value='{value}'max='{max}',style='width: 100%'>{value}
      </progress>
             """.format(loss=loss,value=value, max=max))

In [ ]:
#Sets the module in training mode

num_of_epochs = 15

model.train()

loss_per_10_steps = []
for epoch in range(1,num_of_epochs+1):
    print('Running epoch: {}'.format(epoch))
  
    running_loss=0

    out = display(progress(1, num_of_batches+1), display_id=True)
    for i in range(num_of_batches):
        inputbatch = []
        labelbatch = []
        new_df = df_train[i*batch_size:i*batch_size+batch_size]
        for indx,row in new_df.iterrows():
            inputs = row['input_text']+'</s>' 
            labels = row['target_text']+'</s>'   
            inputbatch.append(inputs)
            labelbatch.append(labels)
        inputbatch = tokenizer.batch_encode_plus(inputbatch,padding=True,truncation=True,return_tensors='pt')["input_ids"]
        labelbatch = tokenizer.batch_encode_plus(labelbatch,padding=True,truncation=True,return_tensors="pt")["input_ids"]
        inputbatch = inputbatch.to(dev)
        labelbatch = labelbatch.to(dev)

        # clear out the gradients of all Variables 
        optimizer.zero_grad()

        # Forward propogation
        outputs = model(input_ids=inputbatch, labels=labelbatch)
        loss = outputs.loss
        loss_num=loss.item()
        logits = outputs.logits
        running_loss+=loss_num
        if i%10 ==0:      
            loss_per_10_steps.append(loss_num)
        out.update(progress(loss_num,i, num_of_batches+1))

        # calculating the gradients
        loss.backward()

        #updating the params
        optimizer.step()

    running_loss=running_loss/int(num_of_batches)
    print('Epoch: {} , Running loss: {}'.format(epoch,running_loss))


Running epoch: 1


/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5.py:226: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


Epoch: 1 , Running loss: 1.0054126112443496
Running epoch: 2


Epoch: 2 , Running loss: 0.9616369956973734
Running epoch: 3


Epoch: 3 , Running loss: 0.9205098451283585
Running epoch: 4


...

Epoch: 11 , Running loss: 0.6477029707928865
Running epoch: 12


Epoch: 12 , Running loss: 0.6241181062816253
Running epoch: 13


Epoch: 13 , Running loss: 0.5957367826541852
Running epoch: 14


Epoch: 14 , Running loss: 0.5730258386953291
Running epoch: 15


Epoch: 15 , Running loss: 0.5488903985472183


In [ ]:
# torch.save(model.state_dict(),'pytorch_model.bin')

In [ ]:
df_test.head()

,input_text,target_text
0,"table_5 Link prediction results on the test-I,...",Results. Table 5 show the results in the filte...
1,table_3 Comparison on validation perplexity. B...,"In Figure 3, we present the validation perplex..."
2,table_4 Performance on maximally covered datas...,5.1 Maximum coverage comparison. Table 4 shows...
3,table_4 Open-ended and multiple-choice (MC) re...,4.4 Comparison to State-of-the-Art. Table 4 co...
4,table_1 Corpus size (length in token) and syst...,Results. From the whole corpus of 300 sentence...


In [ ]:
input_text = df_test['input_text'][18]
input_text

'table_5 Performance on monolingual word similarity computation with seed lexicon size 6000. Method BiLex Method CLSP-WR Method CLSP-SE Chinese (source) WS-240 Chinese (source) WS-297 English (target) WS-353 English (target) SL-999 accuracy accuracy accuracy accuracy 60.36 62.17 60.46 27.22 61.27 65.25 60.46 27.22 60.84 65.62 62.47 28.79'

In [ ]:

model.to(dev)

input_ids = tokenizer.encode(input_text, truncation=True, return_tensors='pt')
input_ids = input_ids.to(dev)
output = model.generate(input_ids=input_ids, max_length=512)
tokenizer.decode(output[0])

'<pad> Table 5 shows the performance of a CLSP-SE model on a monolingual word similarity. We can see that CLSP-WR performs well on a language model with Japanese (source) and Chinese (source) on the Chinese dataset (see the SL-999 dataset). It is also noticeable that on SL-999 (English) and SL-999 ( English), CLSP-SE performs better than T-Ls (WS-297) in SL-999 (WS-297 ( English).</s>'

## Playground

In [ ]:
import transformers
from datasets import load_dataset, load_metric

In [ ]:
dataset = load_dataset("yelp_review_full")

Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset yelp_review_full downloaded and prepared to /root/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})